**Write the names of all group members (max. 5 members)**:
- Ramon, Kaspar
- Cyrill, Stotz

When submitting your work, please follow closely the template below.

# Exercise 1

### Question 1.a

Check that $ S_t = s_0 \exp \left( \sigma W_t - \frac{1}{2} \sigma^2 t \right)$ solves the Black-Scholes SDE.

### Question 1.b - Generate dataset

In [2]:
# Generate the training set

# Generate the test set

### Question 1.c - Model implementation

In [3]:
# Implement the deep hedging model

# You may use and edit the code shown in demo.ipynb

### Question 1.d - Model training

In [2]:
# Implement the payoff function of a call option

# Implement a function that returns the Black-Scholes price of a call option with given strike and maturity

# Train the model

### Question 1.e - Model evaluation

In [5]:
# Evaluate the hedging losses on the test set

# Plot a histogram of the hedging losses and print mean and standard deviation

A few questions for you to consider...

Have you trained your model well?
- Are the losses centered on zero?
- Is their standard deviation sufficiently small?

How can you improve performance?
- train for more epochs
- try different batch sizes
- try a different learning rate
- use a learning rate scheduler (see https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)
- try a different activation function
- change the networks' architectures (more/less hidden neurons, more/less hidden layers)
- reduce/increase the learning rate
- implement regularization (use the `weight_decay` argument in the optimizer)
- implement batch-normalization (see https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html)
- experiment with different optimizers
- ask in the forum

# Exercise 2

### Question 2.a - Compute analytical solution

Show a **step-by-step** derivation of the analytical hedging strategy:

\begin{align*}
H^{\text{BS}}_t(s) & = \frac{\partial C(s, t)}{\partial s} \\
& = \ldots
\end{align*}

### Question 2.b - Analytical solution evaluation

In [6]:
# Implement the formula you found

# Evaluate the losses of the analytical hedging strategies on the test set

**Something to think about**: in the Black-Scholes model every option can be perfectly hedged using its analytical hedging strategy.
But the analytical solution you have computed does not give zero losses. Can you explain why?

### Question 2.c - Hedging losses comparison

How does your deep hedging model compare with the analytical solution?
- If it's worse, you must improve its performance (see checklist above)

### Question 2.d - Learned strategy comparison

In [3]:
# Plot both the neural network strategies and the analytical hedging strategies

- For what times $t_j$ are the two functions most similar? 
- Why?

# Exercise 3

### Question 3.a - Model implementation

In [4]:
# Implement the new deep hedging model

### Question 3.b - Model training

In [6]:
# Train the new deep hedging model

### Question 3.c - Model evaluation

In [8]:
# Evaluate the hedging losses on the test set

# Plot a histogram of the hedging losses and print mean and standard deviation

### Question 3.d - Model comparison

In [9]:
# Compare with the first deep hedging model (Exercise 1)
# Keep the comparison fair: evaluate run-time, performance and number of parameters.